In [5]:
sc

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-0>

In [6]:
import time
from pyspark.sql import SparkSession
import pyspark.sql.functions as F


spark = SparkSession \
    .builder \
    .appName("readCsv") \
    .getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
start = time.time()

df = spark.read.option("header","true").csv('s3://sparkbucket123/1500000 Sales Records.csv')
customSchema = df.schema
cnt_cond = lambda cond: F.sum(F.when(cond, 1).otherwise(0))
df_agg = df.filter(F.col("Region") == "Europe" ).groupby("Country").agg(cnt_cond(F.col('Sales Channel') == 'Offline' ).alias('offline_transactions'), cnt_cond(F.col('Sales Channel') == 'Online' ).alias('online_transactions'))
df_sort = df_agg.sort(F.col("offline_transactions").desc())
df_sort.show(3)

print("CSV data load Time took is {0}".format(time.time() - start))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------------------+-------------------+
|    Country|offline_transactions|online_transactions|
+-----------+--------------------+-------------------+
|     Norway|                4156|               4039|
|Netherlands|                4106|               4052|
|     Russia|                4097|               3990|
+-----------+--------------------+-------------------+
only showing top 3 rows

CSV data load Time took is 22.606258392333984

In [8]:
start = time.time()

df = spark.read.option("header","true").parquet('s3://sparkbucket123/sales_data.parquet')
cnt_cond = lambda cond: F.sum(F.when(cond, 1).otherwise(0))
df_agg = df.filter(F.col("Region") == "Europe" ).groupby("Country").agg(cnt_cond(F.col('Sales_Channel') == 'Offline' ).alias('offline_transactions'), cnt_cond(F.col('Sales_Channel') == 'Online' ).alias('online_transactions'))
df_sort = df_agg.sort(F.col("offline_transactions").desc())
df_sort.show(3)

print("Parquet data load Time took is {0}".format(time.time() - start))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------------------+-------------------+
|    Country|offline_transactions|online_transactions|
+-----------+--------------------+-------------------+
|     Norway|                4156|               4039|
|Netherlands|                4106|               4052|
|     Russia|                4097|               3990|
+-----------+--------------------+-------------------+
only showing top 3 rows

Parquet data load Time took is 4.060359954833984